In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
[beta1,beta2,beta3,beta4] = np.around(np.random.uniform(low=0.01, high=0.2, size=(4,)),4)
beta4


0.1014

In [3]:
#Food_safety_ABM

import numpy as np

class Farm:
    def __init__(self, id, technology_level, initial_contamination_rate, region_mean_contamination_rate):
        self.id = id
        self.technology_level = technology_level
        self.contamination_rate = initial_contamination_rate
        self.region_mean_contamination_rate = region_mean_contamination_rate
        self.cost_effort = 0
        self.cost_technology = 0
        self.penalties = [0, 0, 0, 0, 0]
        self.testing_regimes = [0, 0, 0, 0]
        
    def update_contamination_rate(self, c, k, t):
        # calculate new contamination rate based on Eq. (3)
        c_j_t = c[self.id][t]
        k_j_t = k[self.id][t]
        alpha_j_t = np.exp(-c_j_t * k_j_t)
        self.contamination_rate = alpha_j_t * self.region_mean_contamination_rate
        
    def calculate_cost(self, f, beta, P, c_e, c_k):
        # calculate total cost function based on Eq. (4)
        f1, f2, f3, f4, f5 = f
        beta1, beta2, beta3, beta4 = beta
        numerator = f1 * beta1 + f2 * (1 - beta1) * beta2 + f3 * (1 - beta1) * (1 - beta2) * beta3 + \
                    f4 * (1 - beta1) * (1 - beta2) * (1 - beta3) * beta4 + f5 * (1 - beta1) * (1 - beta2) * \
                    (1 - beta3) * (1 - beta4) * P
        denominator = (c_e + c_k) * (1 - beta1) * (1 - beta2) * (1 - beta3) * (1 - beta4)
        self.cost_effort = c_e
        self.cost_technology = c_k
        self.penalties = [f1, f2, f3, f4, f5]
        self.testing_regimes = [beta1, beta2, beta3, beta4]
        return numerator / denominator
        
    def update_risk_control_behavior(self, f, beta, P, c_e, c_k, c, k, t, neighbors):
        # update risk control behavior based on Eq. (5) and the behavior of neighboring farms
        cost = self.calculate_cost(f, beta, P, c_e, c_k)
        for neighbor in neighbors:
            cost += neighbor.calculate_cost(f, beta, P, c_e, c_k)
        self.cost_effort = c_e
        self.cost_technology = c_k
        self.penalties = [f1, f2, f3, f4, f5]
        self.testing_regimes = [beta1, beta2, beta3, beta4]
        self.contamination_rate = self.region_mean_contamination_rate
        self.contamination_rate += np.random.normal(loc=0, scale=self.technology_level)
        self.contamination_rate += np.random.uniform(low=-1, high=1) * c[self.id][t]
        self.contamination_rate += np.sum([np.random.uniform(low=-1, high=1) * neighbor.contamination_rate for neighbor in neighbors])
        self.update_contamination_rate(c, k, t)
        self.cost = self.calculate_cost(f, beta, P, c_e, c_k)


class FarmSimulation:
	def init(self, n_farms, n_timesteps, technology_levels, initial_contamination_rates, region_mean_contamination_rates):
		self.n_farms = n_farms
		self.n_timesteps = n_timesteps
		self.technology_levels = technology_levels
		self.initial_contamination_rates = initial_contamination_rates
		self.region_mean_contamination_rates = region_mean_contamination_rates
		self.farms = []
		self.c = np.random.normal(loc=0, scale=1, size=(n_farms, n_timesteps))
		self.k = np.random.normal(loc=0, scale=1, size=(n_farms, n_timesteps))
		self.P = 0
		for i in range(n_farms):
			farm = Farm(i, technology_levels[i], initial_contamination_rates[i], region_mean_contamination_rates[i])
			self.farms.append(farm)
	def run_simulation(self, f, beta, c_e, c_k):
     for t in range(self.n_timesteps):
         for i in range(self.n_farms):
             farm = self.farms[i]
             neighbors = self.get_neighbors(i)
             farm.update_risk_control_behavior(f, beta, self.P, c_e, c_k, self.c, self.k, t, neighbors)
             self.P += 0.01	  
             
	def get_neighbors(self, farm_id):
	    # get neighboring farms for a given farm
	    # for simplicity, we assume that farms are arranged in a grid and have four neighbors (north, south, east, west)
	    row = farm_id // int(np.sqrt(self.n_farms))
	    col = farm_id % int(np.sqrt(self.n_farms))
	    neighbors = []
	    if row > 0:
	        neighbors.append(self.farms[(row-1) * int(np.sqrt(self.n_farms)) + col])
	    if row < int(np.sqrt(self.n_farms))-1:
	        neighbors.append(self.farms[(row+1) * int(np.sqrt(self.n_farms)) + col])
		if col > 0:
			neighbors.append(self.farms[row * int(np.sqrt(self.n_farms)) + col-1])
		if col < int(np.sqrt(self.n_farms))-1:
			neighbors.append(self.farms[row * int(np.sqrt(self.n_farms)) + col+1])
		return neighbors

	def plot_contamination_rates(self):
	    # plot contamination rates of all farms over time
	    contamination_rates = []
	    for i in range(self.n_farms):
	        farm = self.farms[i]
	        farm_contamination_rates = [farm.contamination_rate]
	        for t in range(1, self.n_timesteps):
	            farm.update_contamination_rate(self.c, self.k, t)
	            farm_contamination_rates.append(farm.contamination_rate)
	        contamination_rates.append(farm_contamination_rates)
	    plt.figure(figsize=(10,6))
	    plt.title('Contamination Rates Over Time')
	    plt.xlabel('Time')
	    plt.ylabel('Contamination Rate')
	    for i in range(self.n_farms):
	        plt.plot(contamination_rates[i])
	    plt.show()

from ABM_model import FarmSimulation

# Define simulation parameters
n_farms = 100
n_timesteps = 100
technology_levels = [1] * n_farms
initial_contamination_rates = [0.5] * n_farms
region_mean_contamination_rates = [0.5] * n_farms
f = [0.2, 0.2, 0.2, 0.2, 0.2]
beta = [0.25, 0.25, 0.25, 0.25]
c_e = 0.1
c_k = 0.1

# Create and run simulation
simulation = FarmSimulation()
simulation.init(n_farms, n_timesteps, technology_levels, initial_contamination_rates, region_mean_contamination_rates)
for t in range(n_timesteps):
    simulation.run_simulation(f, beta, c_e, c_k)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 66)